**Web Scraping**

In [119]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import ISRIStemmer
import spacy


In [120]:
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client["mydatabase"]
collection = db["mycollection"]

# URL of the website to scrape
url = "https://www.aljazeera.net/news/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all article elements
articles = soup.find_all('article')
# Extract title, link, and summary for each article
arabic_articles = []
for article in articles:
    title = article.find('h3').text
    link = article.find('a')['href']
    summary = article.find('p').text
    arabic_articles.append({'title': title, 'link': link, 'summary': summary})

# Insert articles into MongoDB
collection.insert_many(arabic_articles)

InsertManyResult([ObjectId('66132fd810b802be0045d8ea'), ObjectId('66132fd810b802be0045d8eb'), ObjectId('66132fd810b802be0045d8ec'), ObjectId('66132fd810b802be0045d8ed'), ObjectId('66132fd810b802be0045d8ee'), ObjectId('66132fd810b802be0045d8ef'), ObjectId('66132fd810b802be0045d8f0'), ObjectId('66132fd810b802be0045d8f1'), ObjectId('66132fd810b802be0045d8f2'), ObjectId('66132fd810b802be0045d8f3'), ObjectId('66132fd810b802be0045d8f4'), ObjectId('66132fd810b802be0045d8f5'), ObjectId('66132fd810b802be0045d8f6'), ObjectId('66132fd810b802be0045d8f7')], acknowledged=True)

**NLP Pipeline**

In [109]:
import os
import spacy
from nltk.stem import ISRIStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# Example Arabic sentence
arabic_text = "القطط تحب الأسماك وتحب اللعب أيضا"

# Tokenize the Arabic text
tokens = word_tokenize(arabic_text)

# Apply stemming using ISRIStemmer
stemmer = ISRIStemmer()
stemmed_tokens = [stemmer.stem(token) for token in tokens]

# Apply lemmatization using NLTK WordNetLemmatizer
lemmatizer_nltk = WordNetLemmatizer()
lemmatized_tokens_nltk = [lemmatizer_nltk.lemmatize(token) for token in tokens]

# Initialize Spacy pipeline for Arabic
nlp_spacy = spacy.blank('ar')

# Tokenize and lemmatize the Arabic text using Spacy
doc_spacy = nlp_spacy(arabic_text)
lemmatized_tokens_spacy = [token.lemma_ for token in doc_spacy]

# POS Tagging using NLTK
pos_tags_nltk = pos_tag(tokens)

print("Original Tokens:", tokens)
print("Stemmed Tokens (ISRIStemmer):", stemmed_tokens)
print("Lemmatized Tokens (NLTK WordNetLemmatizer):", lemmatized_tokens_nltk)
print("Lemmatized Tokens (Spacy):", lemmatized_tokens_spacy)
print("POS Tags (NLTK):", pos_tags_nltk)


Original Tokens: ['القطط', 'تحب', 'الأسماك', 'وتحب', 'اللعب', 'أيضا']
Stemmed Tokens (ISRIStemmer): ['قطط', 'تحب', 'سمك', 'تحب', 'لعب', 'ايض']
Lemmatized Tokens (NLTK WordNetLemmatizer): ['القطط', 'تحب', 'الأسماك', 'وتحب', 'اللعب', 'أيضا']
Lemmatized Tokens (Spacy): ['', '', '', '', '', '']
POS Tags (NLTK): [('القطط', 'JJ'), ('تحب', 'NNP'), ('الأسماك', 'NNP'), ('وتحب', 'NNP'), ('اللعب', 'NNP'), ('أيضا', 'NN')]


In [110]:
arabic_articles = collection.find()

for article in arabic_articles:
    normalized_tokens = nlp_pipeline(article['summary'])  # nb: I chosen summary
    
    collection.update_one(
        {'_id': article['_id']},
        {'$set': {
            'normalized_tokens': normalized_tokens  
        }}
    )

In [111]:
#  Speech technics based on both Rule based and Machine learning approaches.
from nltk import pos_tag
# Example Arabic text :test
arabic_text = "انفجار يهز مدينة بيروت ويثير الذعر في الشوارع"
arabic_tokens = word_tokenize(arabic_text)

# Perform POS tagging
arabic_pos_tags = pos_tag(arabic_tokens)
print("Rule-Based - NLTK:", arabic_pos_tags)

Rule-Based - NLTK: [('انفجار', 'JJ'), ('يهز', 'NNP'), ('مدينة', 'NNP'), ('بيروت', 'NNP'), ('ويثير', 'NNP'), ('الذعر', 'NNP'), ('في', 'NNP'), ('الشوارع', 'NN')]


In [112]:
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client["mydatabase"]
collection = db["mycollection"]

# Retrieve articles from the collection
arabic_articles = collection.find()
for article in arabic_articles:
    text = article.get('summary', '')
    
    # Tokenize the Arabic text into words
    arabic_tokens = word_tokenize(text)
    
    # Perform POS tagging
    arabic_pos_tags = pos_tag(arabic_tokens)
    
    # Print POS tags for the article
    print("Article Title:", article.get('title'))
    print("POS Tags (Rule-Based - NLTK):", arabic_pos_tags)
    print("---------------------------------------------------")

Article Title: الحرب على غزة .. شهداء في قصف للاحتلال ونتنياهو يوسع صلاحيات وفد التفاوض
POS Tags (Rule-Based - NLTK): [('مع', 'JJ'), ('دخول', 'NNP'), ('العدوان', 'NNP'), ('الإسرائيلي', 'NNP'), ('على', 'NNP'), ('قطاع', 'NNP'), ('غزة', 'NNP'), ('شهره', 'NNP'), ('السابع', 'NNP'), ('وفي', 'NNP'), ('اليوم', 'NNP'), ('الـ', 'VBD'), ('185،', 'CD'), ('استشهد', 'JJ'), ('7', 'CD'), ('فلسطينيين', 'JJ'), ('في', 'NNP'), ('قصف', 'NNP'), ('للاحتلال', 'NNP'), ('وسط', 'NNP'), ('القطاع', 'NNP'), ('ومخيم', 'NNP'), ('الشجاعية', 'NNP'), ('.', '.')]
---------------------------------------------------
Article Title: 13 شهيدا في غزة وجيش الاحتلال يقر بمقتل وإصابة 12 من عناصره
POS Tags (Rule-Based - NLTK): [('استشهد', 'RB'), ('13', 'CD'), ('فلسطينيا', 'JJ'), ('على', 'NNP'), ('الأقل', 'NNP'), ('وأصيب', 'NNP'), ('آخرون،', 'NNP'), ('مساء', 'NNP'), ('اليوم', 'NNP'), ('الأحد،', 'NNP'), ('إثر', 'NNP'), ('استهداف', 'NNP'), ('قوات', 'NNP'), ('الاحتلال', 'NNP'), ('الإسرائيلي', 'NNP'), ('لمناطق', 'NNP'), ('مختلفة', 'NNP

In [113]:
#the machine learning-based POS tagging
#the pre-trained spaCy model for multilingual (so for arabic too)
nlp = spacy.load('xx_ent_wiki_sm')
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client["mydatabase"]
collection = db["mycollection"]

# Retrieve articles from the collection
arabic_articles = collection.find()
# Iterate over each article and perform POS tagging
for article in arabic_articles:
    
    text = article.get('summary', '') 
    doc = nlp(text)
    arabic_pos_tags = [(token.text, token.pos_) for token in doc]

    print("Article Title:", article.get('title'))
    print("POS Tags :Machine Learning Approach :", arabic_pos_tags)
    print("---------------------------------------------------")

Article Title: الحرب على غزة .. شهداء في قصف للاحتلال ونتنياهو يوسع صلاحيات وفد التفاوض
POS Tags :Machine Learning Approach : [('مع', ''), ('دخول', ''), ('العدوان', ''), ('الإسرائيلي', ''), ('على', ''), ('قطاع', ''), ('غزة', ''), ('شهره', ''), ('السابع', ''), ('وفي', ''), ('اليوم', ''), ('الـ', ''), ('185', ''), ('،', ''), ('استشهد', ''), ('7', ''), ('فلسطينيين', ''), ('في', ''), ('قصف', ''), ('للاحتلال', ''), ('وسط', ''), ('القطاع', ''), ('ومخيم', ''), ('الشجاعية', ''), ('.', '')]
---------------------------------------------------
Article Title: 13 شهيدا في غزة وجيش الاحتلال يقر بمقتل وإصابة 12 من عناصره
POS Tags :Machine Learning Approach : [('استشهد', ''), ('13', ''), ('فلسطينيا', ''), ('على', ''), ('الأقل', ''), ('وأصيب', ''), ('آخرون', ''), ('،', ''), ('مساء', ''), ('اليوم', ''), ('الأحد', ''), ('،', ''), ('إثر', ''), ('استهداف', ''), ('قوات', ''), ('الاحتلال', ''), ('الإسرائيلي', ''), ('لمناطق', ''), ('مختلفة', ''), ('بقطاع', ''), ('غزة', ''), ('،', ''), ('فيما', ''), ('أعلن', '

In [114]:
#test :
from nltk import ne_chunk


arabic_text = "انفجار يهز مدينة بيروت ويثير الذعر في الشوارع"

arabic_tokens = word_tokenize(arabic_text)

arabic_pos_tags = pos_tag(arabic_tokens)

# Perform NER using NLTK's ne_chunk
arabic_ner_tags = ne_chunk(arabic_pos_tags)

print("NER Tags (Rule-Based - NLTK):", arabic_ner_tags)

NER Tags (Rule-Based - NLTK): (S
  (ORGANIZATION انفجار/JJ)
  (ORGANIZATION يهز/NNP)
  مدينة/NNP
  بيروت/NNP
  ويثير/NNP
  الذعر/NNP
  في/NNP
  الشوارع/NN)


In [115]:
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client["mydatabase"]
collection = db["mycollection"]
# Retrieve articles from the collection
arabic_articles = collection.find()

for article in arabic_articles:
    article_text = article.get('summary', '') 
    arabic_tokens = word_tokenize(article_text)
    arabic_pos_tags = pos_tag(arabic_tokens)
    
    # NER
    arabic_ner_tags = ne_chunk(arabic_pos_tags)

    print("NER Tags for Article:", article['title'])
    print(arabic_ner_tags)
    print("---------------------------------------------------")

NER Tags for Article: الحرب على غزة .. شهداء في قصف للاحتلال ونتنياهو يوسع صلاحيات وفد التفاوض
(S
  مع/JJ
  (ORGANIZATION دخول/NNP)
  العدوان/NNP
  الإسرائيلي/NNP
  على/NNP
  قطاع/NNP
  غزة/NNP
  شهره/NNP
  السابع/NNP
  وفي/NNP
  اليوم/NNP
  الـ/VBD
  185،/CD
  استشهد/JJ
  7/CD
  (ORGANIZATION فلسطينيين/JJ)
  في/NNP
  قصف/NNP
  للاحتلال/NNP
  وسط/NNP
  القطاع/NNP
  ومخيم/NNP
  الشجاعية/NNP
  ./.)
---------------------------------------------------
NER Tags for Article: 13 شهيدا في غزة وجيش الاحتلال يقر بمقتل وإصابة 12 من عناصره
(S
  استشهد/RB
  13/CD
  (ORGANIZATION فلسطينيا/JJ)
  (ORGANIZATION على/NNP)
  الأقل/NNP
  وأصيب/NNP
  آخرون،/NNP
  مساء/NNP
  اليوم/NNP
  الأحد،/NNP
  إثر/NNP
  استهداف/NNP
  قوات/NNP
  الاحتلال/NNP
  الإسرائيلي/NNP
  لمناطق/NNP
  مختلفة/NNP
  بقطاع/NNP
  غزة،/NNP
  فيما/NNP
  أعلن/NNP
  جيش/NNP
  الاحتلال/NNP
  مقتل/NNP
  وإصابة/VBD
  12/CD
  من/NNP
  عناصره/NNP
  في/NNP
  المعارك/NNP
  الدائرة/NNP
  بالقطاع/NNP
  ./.)
-----------------------------------------

In [116]:
#spaCy (Machine Learning-Based):
# Load the Arabic spaCy model
nlp = spacy.blank('ar')

# Example Arabic text
arabic_text = "انفجار يهز مدينة بيروت ويثير الذعر في الشوارع"

# Process the text using spaCy
doc = nlp(arabic_text)

# Extract named entities using spaCy
arabic_ner_tags = [(ent.text, ent.label_) for ent in doc.ents]

print("NER Tags (Machine Learning-Based - spaCy):", arabic_ner_tags)

NER Tags (Machine Learning-Based - spaCy): []


In [118]:
nlp = spacy.load('xx_ent_wiki_sm')  
client = MongoClient('mongodb://localhost:27017/')
db = client["mydatabase"]
collection = db["mycollection"]
# Retrieve articles from the collection
arabic_articles = collection.find()
for article in arabic_articles:
   
    article_text = article.get('summary', '')  
    doc = nlp(article_text)
    

    print("NER Tags :Machine Learning Approach for Article:", article['title'])
    for ent in doc.ents:
        print(ent.text, ent.label_)
    print("---------------------------------------------------")

NER Tags :Machine Learning Approach for Article: الحرب على غزة .. شهداء في قصف للاحتلال ونتنياهو يوسع صلاحيات وفد التفاوض
---------------------------------------------------
NER Tags :Machine Learning Approach for Article: 13 شهيدا في غزة وجيش الاحتلال يقر بمقتل وإصابة 12 من عناصره
---------------------------------------------------
NER Tags :Machine Learning Approach for Article: بعد انسحاب جيش الاحتلال من خان يونس.. الدمار ورائحة الموت في كل مكان
---------------------------------------------------
NER Tags :Machine Learning Approach for Article: القدس.. نقل الدكتور ناجح بكيرات إلى المستشفى فور الإفراج عنه من سجون الاحتلال
---------------------------------------------------
NER Tags :Machine Learning Approach for Article: القسام لجيش الاحتلال: كلما دخلتم غزة ستحترقون في أزقتها
مشاهد PER
---------------------------------------------------
NER Tags :Machine Learning Approach for Article: اتهامات للدعم السريع بارتكاب 3 مجازر في السودان
---------------------------------------------------
